In [1]:
import os
import glob
import pickle
import ntpath
import timeit
import numpy as np
import cv2 as cv

from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from copy import deepcopy
from skimage.feature import hog

In [2]:
# Crop images

# file_barney = open('antrenare/wilma_annotations.txt', 'r')
# Lines = file_barney.readlines()
# 
# for idx, line in enumerate(Lines):
#     temp = line.split(' ')
#     img = cv.imread('antrenare/wilma/'+temp[0])
#     xmin = int(temp[1])
#     ymin = int(temp[2])
#     xmax = int(temp[3])
#     ymax = int(temp[4])   
#     y = ymax-ymin
#     x = xmax-xmin
#     if(y > x):
#         cropped_image = img[ymin:ymax, xmin-(y-x)//2:xmax+(y-x)//2]
#     else:
#         cropped_image = img[ymin-(x-y)//2:ymax+(x-y)//2, xmin:xmax]
#     try:
#         resized_image = cv.resize(cropped_image, (85, 85))
#         cv.imwrite('antrenare/exemple_pozitive/wilma_'+str(idx)+'.jpg', resized_image)
#     except Exception as e:
#         print(str(e))
# 
#     # cv.imshow('image', np.uint8(img))
#     # cv.imshow('image_crop', np.uint8(cropped_image))
# 
#     cv.waitKey(0)

In [3]:
class Parameters:
    def __init__(self):
        self.base_dir = '../Proiect2'
        # path-urile acestea devin redundante daca nu trebuie sa generam modelul
        self.dir_pos_examples = os.path.join(self.base_dir, 'antrenare/exemple_pozitive')
        self.dir_neg_examples = os.path.join(self.base_dir, 'antrenare/exemple_negative')
        
        # path-ul pentru imaginire de testare
        self.dir_test_examples = os.path.join(self.base_dir,'testare')
        
        # path-ul pentru rezultatele corecte ale imaginilor de testare
        self.path_annotations = os.path.join(self.base_dir, 'validare/validare_annotations.txt')
        
        # path-ul pentru folder-ul cu modelele deja generate
        self.dir_save_files = os.path.join(self.base_dir, 'salveazaFisiere')
        
        # path-ul pentru rezultatele finale
        self.dir_result = os.path.join(self.base_dir, 'result')
        
        if not os.path.exists(self.dir_save_files):
            os.makedirs(self.dir_save_files)
            print('directory created: {} '.format(self.dir_save_files))
        else:
            print('directory {} exists '.format(self.dir_save_files))

        # set the parameters
        self.dim_window = 85  # exemplele pozitive (fete de oameni cropate) au 85x85 pixeli
        self.dim_hog_cell = 6  # dimensiunea celulei
        self.dim_descriptor_cell = 85  # dimensiunea descriptorului unei celule
        self.overlap = 0.5
        self.number_positive_examples = 6881  # numarul exemplelor pozitive
        self.number_negative_examples = 4000  # numarul exemplelor negative
        self.overlap = 0.5
        self.has_annotations = False
        self.threshold = 0


In [4]:
def show_detections_without_ground_truth(detections, scores, file_names, params: Parameters):

    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)

    for test_file in test_files:
        image = cv.imread(test_file)
        short_file_name = ntpath.basename(test_file)
        indices_detections_current_image = np.where(file_names == short_file_name)
        current_detections = detections[indices_detections_current_image]
        current_scores = scores[indices_detections_current_image]

        for idx, detection in enumerate(current_detections):
            cv.rectangle(image, (detection[0], detection[1]), (detection[2], detection[3]), (0, 0, 255), thickness=1)
            cv.putText(image, 'score:' + str(current_scores[idx])[:4], (detection[0], detection[1]),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv.imwrite(os.path.join(params.dir_save_files, "detections_" + short_file_name), image)
        print('Apasa orice tasta pentru a continua...')
        cv.imshow('image', np.uint8(image))
        cv.waitKey(0)

def show_detections_with_ground_truth(detections, scores, file_names, params: Parameters):

    ground_truth_bboxes = np.loadtxt(params.path_annotations, dtype='str')
    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)

    for test_file in test_files:
        image = cv.imread(test_file)
        short_file_name = ntpath.basename(test_file)
        indices_detections_current_image = np.where(file_names == short_file_name)
        current_detections = detections[indices_detections_current_image]
        current_scores = scores[indices_detections_current_image]

        for idx, detection in enumerate(current_detections):
            cv.rectangle(image, (detection[0], detection[1]), (detection[2], detection[3]), (0, 0, 255), thickness=1)
            cv.putText(image, 'score:' + str(current_scores[idx])[:4], (detection[0], detection[1]),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        annotations = ground_truth_bboxes[ground_truth_bboxes[:, 0] == short_file_name]

        # show ground truth bboxes
        for detection in annotations:
            cv.rectangle(image, (int(detection[1]), int(detection[2])), (int(detection[3]), int(detection[4])), (0, 255, 0), thickness=1)

        cv.imwrite(os.path.join(params.dir_save_files, "detections_" + short_file_name), image)
        print('Apasa orice tasta pentru a continua...')
        cv.imshow('image', np.uint8(image))
        cv.waitKey(0)
        
def output_file(detections, scores, file_names, add, params):
    if add.strip() == 'all_faces':
        path = os.path.join(params.dir_result,'task1')
    else:
        path = os.path.join(params.dir_result,'task2')
        
    file = os.path.join(path, 'detections_'+add.strip()+'.npy')
    np.save(file, detections)
    
    file = os.path.join(path, 'file_names_'+add.strip()+'.npy')
    np.save(file, file_names)
    
    file = os.path.join(path, 'scores_'+add.strip()+'.npy')
    np.save(file, scores)
            
# def output_file(detections, file, params: Parameters):
#     
#     # MODIFICA!!!
#     file = '../Proiect2/result/'+file
#     f = open(file, "w")
#     
#     test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
#     test_files = glob.glob(test_images_path)
#     for test_file in test_files:
#         image = cv.imread(test_file)
#         short_file_name = ntpath.basename(test_file)
#         indices_detections_current_image = np.where(file_names == short_file_name)
#         current_detections = detections[indices_detections_current_image]
#         current_scores = scores[indices_detections_current_image]
#         
#         for idx, detection in enumerate(current_detections):
#             #(detection[0], detection[1]), (detection[2], detection[3])
#             f.write(short_file_name+" "+str(detection[0])+" "+ str(detection[1])+" "+str(detection[2])+" "+str(detection[3])+"\n")

In [5]:
class FacialDetector:
    def __init__(self, params:Parameters):
        self.params = params
        self.best_model = None

    def get_positive_descriptors(self):
        # in aceasta functie calculam descriptorii pozitivi
        # vom returna un numpy array de dimensiuni NXD
        # unde N - numar exemplelor pozitive
        # iar D - dimensiunea descriptorului
        # D = (params.dim_window/params.dim_hog_cell - 1) ^ 2 * params.dim_descriptor_cell (fetele sunt patrate)

        images_path = os.path.join(self.params.dir_pos_examples, '*.jpg')
        files = glob.glob(images_path)
        num_images = len(files)
        positive_descriptors = []
        print('Calculam descriptorii pt %d imagini pozitive...' % num_images)
        for i in range(num_images):
            print('Procesam exemplul pozitiv numarul %d...' % i)
            img = cv.imread(files[i], cv.IMREAD_GRAYSCALE)
            
            features = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                           cells_per_block=(2, 2), feature_vector=True)
            # print(len(features))

            positive_descriptors.append(features)
            if self.params.use_flip_images:
                features = hog(np.fliplr(img), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                positive_descriptors.append(features)

        positive_descriptors = np.array(positive_descriptors)
        return positive_descriptors

    def get_negative_descriptors(self):
        images_path = os.path.join(self.params.dir_neg_examples, '*.jpg')
        files = glob.glob(images_path)
        num_images = len(files)    
        num_negative_per_image = self.params.number_negative_examples // num_images
        negative_descriptors = []
        print('Calculam descriptorii pt %d imagini negative' % num_images)
        for i in range(num_images):
            print('Procesam exemplul negativ numarul %d...' % i)
            img = cv.imread(files[i], cv.IMREAD_GRAYSCALE)
            
            num_rows = img.shape[0]
            num_cols = img.shape[1]
            print(num_cols - self.params.dim_window," & ", num_rows - self.params.dim_window)
            x = np.random.randint(low=0, high=num_cols - self.params.dim_window, size=num_negative_per_image)
            y = np.random.randint(low=0, high=num_rows - self.params.dim_window, size=num_negative_per_image)

            for idx in range(len(y)):
                patch = img[y[idx]: y[idx] + self.params.dim_window, x[idx]: x[idx] + self.params.dim_window]
                descr = hog(patch, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                            cells_per_block=(2, 2), feature_vector=False)
                negative_descriptors.append(descr.flatten())

        negative_descriptors = np.array(negative_descriptors)
        return negative_descriptors

    def train_classifier(self, training_examples, train_labels, char):
        svm_file_name = os.path.join(self.params.dir_save_files, 'best_model_%d_%d_%d_%s' %
                                     (self.params.dim_hog_cell, self.params.number_negative_examples,
                                      self.params.number_positive_examples, char))
        if os.path.exists(svm_file_name):
            self.best_model = pickle.load(open(svm_file_name, 'rb'))
            return

        best_accuracy = 0
        best_c = 0
        best_model = None
        Cs = [10 ** -5, 10 ** -4,  10 ** -3,  10 ** -2, 10 ** -1, 10 ** 0]
        for c in Cs:
            print('Antrenam un clasificator pentru c=%f' % c)
            model = LinearSVC(C=c)
            model.fit(training_examples, train_labels)
            acc = model.score(training_examples, train_labels)
            print(acc)
            if acc > best_accuracy:
                best_accuracy = acc
                best_c = c
                best_model = deepcopy(model)

        print('Performanta clasificatorului optim pt c = %f' % best_c)
        # salveaza clasificatorul
        pickle.dump(best_model, open(svm_file_name, 'wb'))

        # vizualizeaza cat de bine sunt separate exemplele pozitive de cele negative dupa antrenare
        # ideal ar fi ca exemplele pozitive sa primeasca scoruri > 0, iar exemplele negative sa primeasca scoruri < 0
        scores = best_model.decision_function(training_examples)
        self.best_model = best_model
        positive_scores = scores[train_labels > 0]
        negative_scores = scores[train_labels <= 0]


        plt.plot(np.sort(positive_scores))
        plt.plot(np.zeros(len(positive_scores)))
        plt.plot(np.sort(negative_scores))
        plt.xlabel('Nr example antrenare')
        plt.ylabel('Scor clasificator')
        plt.title('Distributia scorurilor clasificatorului pe exemplele de antrenare')
        plt.legend(['Scoruri exemple pozitive', '0', 'Scoruri exemple negative'])
        plt.show()

    def intersection_over_union(self, bbox_a, bbox_b):
        x_a = max(bbox_a[0], bbox_b[0])
        y_a = max(bbox_a[1], bbox_b[1])
        x_b = min(bbox_a[2], bbox_b[2])
        y_b = min(bbox_a[3], bbox_b[3])
        
        inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)
        
        box_a_area = (bbox_a[2] - bbox_a[0] + 1) * (bbox_a[3] - bbox_a[1] + 1)
        box_b_area = (bbox_b[2] - bbox_b[0] + 1) * (bbox_b[3] - bbox_b[1] + 1)

        if float(box_a_area + box_b_area - inter_area) == 0:
            iou = 0
        else:
            iou = inter_area / float(box_a_area + box_b_area - inter_area)

        return iou

    def non_maximal_suppression(self, image_detections, image_scores, image_size):
        # xmin, ymin, xmax, ymax
        x_out_of_bounds = np.where(image_detections[:, 2] > image_size[1])[0]
        y_out_of_bounds = np.where(image_detections[:, 3] > image_size[0])[0]
        # print(x_out_of_bounds, y_out_of_bounds)
        image_detections[x_out_of_bounds, 2] = image_size[1]
        image_detections[y_out_of_bounds, 3] = image_size[0]
        sorted_indices = np.flipud(np.argsort(image_scores))
        sorted_image_detections = image_detections[sorted_indices]
        sorted_scores = image_scores[sorted_indices]

        is_maximal = np.ones(len(image_detections)).astype(bool)
        iou_threshold = 0.3
        for i in range(len(sorted_image_detections) - 1):
            if is_maximal[i] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                for j in range(i + 1, len(sorted_image_detections)):
                    if is_maximal[j] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                        if self.intersection_over_union(sorted_image_detections[i],sorted_image_detections[j]) > iou_threshold:is_maximal[j] = False
                        else:  # verificam daca centrul detectiei este in mijlocul detectiei cu scor mai mare
                            c_x = (sorted_image_detections[j][0] + sorted_image_detections[j][2]) / 2
                            c_y = (sorted_image_detections[j][1] + sorted_image_detections[j][3]) / 2
                            if sorted_image_detections[i][0] <= c_x <= sorted_image_detections[i][2] and \
                                    sorted_image_detections[i][1] <= c_y <= sorted_image_detections[i][3]:
                                is_maximal[j] = False
        return sorted_image_detections[is_maximal], sorted_scores[is_maximal]

    def run(self):
        test_images_path = os.path.join(self.params.dir_test_examples, '*.jpg')
        test_files = glob.glob(test_images_path)
        detections = None  # array cu toate detectiile pe care le obtinem
        scores = np.array([])  # array cu toate scorurile pe care le obtinem
        file_names = np.array([])  # array cu fisiele, in aceasta lista fisierele vor aparea de mai multe ori, pentru fiecare
        # detectie din imagine, numele imaginii va aparea in aceasta lista
        w = self.best_model.coef_.T
        bias = self.best_model.intercept_[0]
        num_test_images = len(test_files)
        descriptors_to_return = []
        for i in range(num_test_images):
            start_time = timeit.default_timer()
            print('Procesam imaginea de testare %d/%d..' % (i, num_test_images))
            img = cv.imread(test_files[i], cv.IMREAD_GRAYSCALE)
            
            scale_nr = {0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0, 1.25, 1.5}
            image_scores = []
            image_detections = []
            
            for dim in scale_nr:
                img_sliding_win = cv.resize(img, None, fx = dim, fy = dim)
                
                hog_descriptors = hog(img_sliding_win, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                      cells_per_block=(2, 2), feature_vector=False)
                num_cols = img_sliding_win.shape[1] // self.params.dim_hog_cell - 1
                num_rows = img_sliding_win.shape[0] // self.params.dim_hog_cell - 1
                num_cell_in_template = self.params.dim_window // self.params.dim_hog_cell - 1
                
                for y in range(0, num_rows - num_cell_in_template):
                    for x in range(0, num_cols - num_cell_in_template):
                        descr = hog_descriptors[y:y + num_cell_in_template, x:x + num_cell_in_template].flatten()
                        score = np.dot(descr, w)[0] + bias
                        
                        if score > self.params.threshold:
                            x_min = int(x * self.params.dim_hog_cell / dim)
                            y_min = int(y * self.params.dim_hog_cell / dim)
                            x_max = int((x * self.params.dim_hog_cell + self.params.dim_window)/ dim)
                            y_max = int((y * self.params.dim_hog_cell + self.params.dim_window)/ dim)
                            image_detections.append([x_min, y_min, x_max, y_max])
                            image_scores.append(score)
                            
            if len(image_scores) > 0:
                image_detections, image_scores = self.non_maximal_suppression(np.array(image_detections),
                                                                                  np.array(image_scores), img.shape)
                    
            if len(image_scores) > 0:
                if detections is None:
                    detections = image_detections
                else:
                    detections = np.concatenate((detections, image_detections))
                scores = np.append(scores, image_scores)
                short_name = ntpath.basename(test_files[i])
                image_names = [short_name for ww in range(len(image_scores))]
                file_names = np.append(file_names, image_names)

            end_time = timeit.default_timer()
            print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
                  % (i, num_test_images, end_time - start_time))

        return detections, scores, file_names

    def compute_average_precision(self, rec, prec):
        # functie adaptata din 2010 Pascal VOC development kit
        m_rec = np.concatenate(([0], rec, [1]))
        m_pre = np.concatenate(([0], prec, [0]))
        for i in range(len(m_pre) - 1, -1, 1):
            m_pre[i] = max(m_pre[i], m_pre[i + 1])
        m_rec = np.array(m_rec)
        i = np.where(m_rec[1:] != m_rec[:-1])[0] + 1
        average_precision = np.sum((m_rec[i] - m_rec[i - 1]) * m_pre[i])
        return average_precision

    def eval_detections(self, detections, scores, file_names, fig = 'precizie_medie.png'):
        ground_truth_file = np.loadtxt(self.params.path_annotations, dtype='str')
        ground_truth_file_names = np.array(ground_truth_file[:, 0])
        ground_truth_detections = np.array(ground_truth_file[:, 1:5], dtype=int)

        num_gt_detections = len(ground_truth_detections)  # numar total de adevarat pozitive
        gt_exists_detection = np.zeros(num_gt_detections)
        # sorteazam detectiile dupa scorul lor
        sorted_indices = np.argsort(scores)[::-1]
        file_names = file_names[sorted_indices]
        scores = scores[sorted_indices]
        detections = detections[sorted_indices]

        num_detections = len(detections)
        true_positive = np.zeros(num_detections)
        false_positive = np.zeros(num_detections)
        duplicated_detections = np.zeros(num_detections)

        for detection_idx in range(num_detections):
            indices_detections_on_image = np.where(ground_truth_file_names == file_names[detection_idx])[0]

            gt_detections_on_image = ground_truth_detections[indices_detections_on_image]
            bbox = detections[detection_idx]
            max_overlap = -1
            index_max_overlap_bbox = -1
            for gt_idx, gt_bbox in enumerate(gt_detections_on_image):
                overlap = self.intersection_over_union(bbox, gt_bbox)
                if overlap > max_overlap:
                    max_overlap = overlap
                    index_max_overlap_bbox = indices_detections_on_image[gt_idx]

            # clasifica o detectie ca fiind adevarat pozitiva / fals pozitiva
            if max_overlap >= 0.3:
                if gt_exists_detection[index_max_overlap_bbox] == 0:
                    true_positive[detection_idx] = 1
                    gt_exists_detection[index_max_overlap_bbox] = 1
                else:
                    false_positive[detection_idx] = 1
                    duplicated_detections[detection_idx] = 1
            else:
                false_positive[detection_idx] = 1

        cum_false_positive = np.cumsum(false_positive)
        cum_true_positive = np.cumsum(true_positive)

        rec = cum_true_positive / num_gt_detections
        prec = cum_true_positive / (cum_true_positive + cum_false_positive)
        average_precision = self.compute_average_precision(rec, prec)
        plt.plot(rec, prec, '-')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Average precision %.3f' % average_precision)
        plt.savefig(os.path.join(self.params.dir_save_files, fig))
        plt.show()
        

In [6]:
## TASK 1.

params: Parameters = Parameters()
params.dim_window = 85  # exemplele pozitive (fete de oameni cropate) au 85x85 pixeli
params.dim_hog_cell = 6  # dimensiunea celulei
params.overlap = 0.5
params.number_positive_examples = 6881  # numarul exemplelor pozitive
params.number_negative_examples = 4000 # numarul exemplelor negative

params.threshold = 2.5  # toate ferestrele cu scorul > threshold si maxime locale devin detectii
params.has_annotations = True

params.use_hard_mining = False  # (optional)antrenare cu exemple puternic negative
params.use_flip_images = True  # adauga imaginile cu fete oglindite

if params.use_flip_images:
    params.number_positive_examples *= 2

facial_detector: FacialDetector = FacialDetector(params)

# Pasii 1+2+3. Incarcam exemplele pozitive (cropate) si exemple negative generate
# verificam daca sunt deja existente
positive_features_path = os.path.join(params.dir_save_files, 'descriptoriExemplePozitive_' + str(params.dim_hog_cell) + '_' +
                        str(params.number_positive_examples) + '.npy')
if os.path.exists(positive_features_path):
    positive_features = np.load(positive_features_path)
    print('Am incarcat descriptorii pentru exemplele pozitive')
else:
    print('Construim descriptorii pentru exemplele pozitive:')
    positive_features = facial_detector.get_positive_descriptors()
    np.save(positive_features_path, positive_features)
    print('Am salvat descriptorii pentru exemplele pozitive in fisierul %s' % positive_features_path)

# exemple negative
negative_features_path = os.path.join(params.dir_save_files, 'descriptoriExempleNegative_' + str(params.dim_hog_cell) + '_' +
                        str(params.number_negative_examples) + '.npy')
if os.path.exists(negative_features_path):
    negative_features = np.load(negative_features_path)
    print('Am incarcat descriptorii pentru exemplele negative')
else:
    print('Construim descriptorii pentru exemplele negative:')
    negative_features = facial_detector.get_negative_descriptors()
    np.save(negative_features_path, negative_features)
    print('Am salvat descriptorii pentru exemplele negative in fisierul %s' % negative_features_path)

# Pasul 4. Invatam clasificatorul liniar
print(np.squeeze(positive_features).shape,np.squeeze(negative_features).shape)
training_examples = np.concatenate((np.squeeze(positive_features), np.squeeze(negative_features)), axis=0)
train_labels = np.concatenate((np.ones(params.number_positive_examples), np.zeros(negative_features.shape[0])))
text = "task1"
facial_detector.train_classifier(training_examples, train_labels, text)

detections, scores, file_names = facial_detector.run()

# if params.has_annotations:
#     facial_detector.eval_detections(detections, scores, file_names)
#     # show_detections_with_ground_truth(detections, scores, file_names, params)
# else:
#     show_detections_without_ground_truth(detections, scores, file_names, params)
    
# output files
f = 'all_faces'
output_file(detections, scores, file_names, f, params) 

directory ../Proiect2\salveazaFisiere exists 
Am incarcat descriptorii pentru exemplele pozitive
Am incarcat descriptorii pentru exemplele negative
(13762, 6084) (3956, 6084)
Procesam imaginea de testare 0/200..
Timpul de procesarea al imaginii de testare 0/200 este 3.023936 sec.
Procesam imaginea de testare 1/200..
Timpul de procesarea al imaginii de testare 1/200 este 1.753553 sec.
Procesam imaginea de testare 2/200..
Timpul de procesarea al imaginii de testare 2/200 este 1.740601 sec.
Procesam imaginea de testare 3/200..
Timpul de procesarea al imaginii de testare 3/200 este 1.731708 sec.
Procesam imaginea de testare 4/200..
Timpul de procesarea al imaginii de testare 4/200 este 1.764428 sec.
Procesam imaginea de testare 5/200..
Timpul de procesarea al imaginii de testare 5/200 este 1.632511 sec.
Procesam imaginea de testare 6/200..
Timpul de procesarea al imaginii de testare 6/200 este 1.656907 sec.
Procesam imaginea de testare 7/200..
Timpul de procesarea al imaginii de testare 7/

In [7]:
## TASK 2.

params: Parameters = Parameters()
params.dim_window = 85  # exemplele pozitive (fete de oameni cropate) au 85x85 pixeli
params.dim_hog_cell = 6  # dimensiunea celulei
params.overlap = 0.5

params.threshold = 2.5  # toate ferestrele cu scorul > threshold si maxime locale devin detectii
params.has_annotations = True

params.use_hard_mining = False  # (optional)antrenare cu exemple puternic negative
params.use_flip_images = True  # adauga imaginile cu fete oglindite

characters = ['fred','barney','betty','wilma']

for char in characters:
    file_number = glob.glob('../Proiect2/antrenare/antrenare_'+char.strip()+'/*')
    
    params.number_positive_examples = len(file_number) # numarul exemplelor pozitive
    params.number_negative_examples = 6000 # numarul exemplelor negative

    params.dir_pos_examples = 'antrenare/antrenare_'+char.strip()
    params.dir_neg_examples = 'antrenare/exemple_negative_'+char.strip()
    params.path_annotations = 'validare/task2_'+char.strip()+'_gt_validare.txt'

    if params.use_flip_images:
        params.number_positive_examples *= 2

    facial_detector: FacialDetector = FacialDetector(params)

    # Pasii 1+2+3. Incarcam exemplele pozitive (cropate) si exemple negative generate
    # verificam daca sunt deja existente
    positive_features_path = os.path.join(params.dir_save_files, 'descriptoriExemplePozitive_'+char.strip()+'_'+ str(params.dim_hog_cell) + '_' +
                            str(params.number_positive_examples) + '.npy')
    if os.path.exists(positive_features_path):
        positive_features = np.load(positive_features_path)
        print('Am incarcat descriptorii pentru exemplele pozitive')
    else:
        print('Construim descriptorii pentru exemplele pozitive:')
        positive_features = facial_detector.get_positive_descriptors()
        np.save(positive_features_path, positive_features)
        print('Am salvat descriptorii pentru exemplele pozitive in fisierul %s' % positive_features_path)

    # exemple negative
    negative_features_path = os.path.join(params.dir_save_files, 'descriptoriExempleNegative_'+char.strip()+ str(params.dim_hog_cell) + '_' +
                            str(params.number_negative_examples) + '.npy')
    if os.path.exists(negative_features_path):
        negative_features = np.load(negative_features_path)
        print('Am incarcat descriptorii pentru exemplele negative')
    else:
        print('Construim descriptorii pentru exemplele negative:')
        negative_features = facial_detector.get_negative_descriptors()
        np.save(negative_features_path, negative_features)
        print('Am salvat descriptorii pentru exemplele negative in fisierul %s' % negative_features_path)

    # Pasul 4. Invatam clasificatorul liniarDete
    print(np.squeeze(positive_features).shape,np.squeeze(negative_features).shape)
    training_examples = np.concatenate((np.squeeze(positive_features), np.squeeze(negative_features)), axis=0)
    train_labels = np.concatenate((np.ones(params.number_positive_examples), np.zeros(negative_features.shape[0])))
    facial_detector.train_classifier(training_examples, train_labels, char.strip())

    detections, scores, file_names = facial_detector.run()

    # if params.has_annotations:
    #     fig = 'precizie_medie_'+char.strip()+'.png'
    #     facial_detector.eval_detections(detections, scores, file_names , fig)
    #     # show_detections_with_ground_truth(detections, scores, file_names, params)
    # else:
    #     show_detections_without_ground_truth(detections, scores, file_names, params)

    # output files
    output_file(detections, scores, file_names, char.strip(), params)

directory ../Proiect2\salveazaFisiere exists 
Am incarcat descriptorii pentru exemplele pozitive
Am incarcat descriptorii pentru exemplele negative
(3558, 6084) (5190, 6084)
Procesam imaginea de testare 0/200..
Timpul de procesarea al imaginii de testare 0/200 este 2.858940 sec.
Procesam imaginea de testare 1/200..
Timpul de procesarea al imaginii de testare 1/200 este 3.077465 sec.
Procesam imaginea de testare 2/200..
Timpul de procesarea al imaginii de testare 2/200 este 2.021561 sec.
Procesam imaginea de testare 3/200..
Timpul de procesarea al imaginii de testare 3/200 este 2.001030 sec.
Procesam imaginea de testare 4/200..
Timpul de procesarea al imaginii de testare 4/200 este 2.408513 sec.
Procesam imaginea de testare 5/200..
Timpul de procesarea al imaginii de testare 5/200 este 2.339985 sec.
Procesam imaginea de testare 6/200..
Timpul de procesarea al imaginii de testare 6/200 este 1.935884 sec.
Procesam imaginea de testare 7/200..
Timpul de procesarea al imaginii de testare 7/2